# PBR geometry analysis

In [75]:
import laspy
import numpy as np
import os
import open3d as o3d
from joblib import Parallel, delayed
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def read_las_file(file_path):
    
    # read las file
    las_file = laspy.read(file_path)

    # get the point data
    point_data = las_file.points
    # get the x, y, z coordinates
    x = point_data.x
    y = point_data.y
    z = point_data.z
    # get the intensity values
    semantics = point_data.intensity

    # stock the x, y, z, semantics in a numpy array
    x = np.array(x)
    y = np.array(y)
    z = np.array(z)
    semantics = np.array(semantics)

    # get color values from las file
    red = point_data.red
    green = point_data.green
    blue = point_data.blue

    colors = np.array([red, green, blue]).T

    # stack the arrays
    points_semantics_source = np.vstack((x, y, z, semantics)).T

    print(points_semantics_source.shape)

    # assert the length of the arrays
    assert len(x) == len(y) == len(z) == len(semantics), "Length of x, y, z, and intensity arrays do not match."

    # print the numbers of points and semantics
    print(f"Number of points: {len(x)}")
    print(f"Number of semantics: {np.unique(semantics).size}")

    return points_semantics_source, colors




In [ ]:
def save_points_to_las(points, color, filename):
    # Create a new LAS header and file
    header = laspy.LasHeader(point_format=3, version="1.2")
    las_file = laspy.LasData(header)

    # Set coordinates
    las_file.x = points[:, 0]
    las_file.y = points[:, 1]
    las_file.z = points[:, 2]

    # Handle intensity (semantics)
    semantics = points[:, 3].astype(np.int16)  # Promote to signed int
    max_intensity = semantics[semantics != -1].max()

    if len(semantics[semantics == max_intensity]) < len(semantics)/10:
        semantics[semantics == -1] = max_intensity + 1 
        print(f"Max intensity: {max_intensity + 1}")
    else:
        semantics[semantics == -1] = max_intensity 
        print(f"Max intensity: {max_intensity}")
    las_file.intensity = semantics.astype(np.uint16)  # Cast back to uint16

    

    # Set RGB color
    las_file.red = color[:, 0].astype(np.uint16)
    las_file.green = color[:, 1].astype(np.uint16)
    las_file.blue = color[:, 2].astype(np.uint16)

    # Write the LAS file
    las_file.write(filename)




def cluster_filter(points_semantics, eps=0.5, min_samples=10, n_jobs=8):
    semantics_ids = np.unique(points_semantics[:, 3])

    def process_semantic_group(sem_id):
        # Find global indices for this semantic group
        group_indices = np.where(points_semantics[:, 3] == sem_id)[0]
        group_points = points_semantics[group_indices, :3]

        if len(group_points) < min_samples:
            return group_indices  # All treated as outliers

        clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(group_points)
        labels, counts = np.unique(clustering.labels_, return_counts=True)

        # Ignore noise-only cases (no valid clusters)
        if np.all(labels == -1):
            return group_indices

        largest_cluster_label = labels[np.argmax(counts)]
        inliers_local = np.where(clustering.labels_ == largest_cluster_label)[0]
        all_local = np.arange(len(group_indices))
        outliers_local = np.setdiff1d(all_local, inliers_local)

        # Return global indices of outliers
        return group_indices[outliers_local]

    # Parallel loop over semantic IDs
    outlier_indices_all = Parallel(n_jobs=n_jobs)(
        delayed(process_semantic_group)(sid) for sid in semantics_ids
    )

    all_outlier_indices = np.concatenate(outlier_indices_all)
    points_semantics[all_outlier_indices, 3] = -1

    # print the number of outliers
    print(f"Number of outliers from DBSCAN filter: {len(all_outlier_indices)}")

    return points_semantics



def size_filter(points_semantics, min_horizontal_length=0.5, max_horizontal_length=5.0, min_vertical_length=0.5, max_vertical_length=5.0):
    semantics_ids = np.unique(points_semantics[:, 3])

    def process_semantic_group(points_semantics, semantics_id, min_horizontal_length, max_horizontal_length, min_vertical_length, max_vertical_length):
        # Get indices of points with this semantics
        indices = np.where(points_semantics[:, 3] == semantics_id)[0]
        local_points_semantics = points_semantics[indices]
        xyz = local_points_semantics[:, :3]

        # Calculate the bounding box
        min_x, min_y, min_z = np.min(xyz, axis=0)
        max_x, max_y, max_z = np.max(xyz, axis=0)

        # Calculate lengths
        horizontal_length = np.sqrt((max_x - min_x) ** 2 + (max_y - min_y) ** 2)
        vertical_length = max_z - min_z

        # return True if the lengths are within the specified range; otherwise, return False
        if min_horizontal_length <= horizontal_length <= max_horizontal_length and min_vertical_length <= vertical_length <= max_vertical_length:
            return semantics_id, True
        else:
            return semantics_id, False
        
    # Parallel processing
    results = Parallel(n_jobs=8)(
        delayed(process_semantic_group)(points_semantics, sid, min_horizontal_length, max_horizontal_length, min_vertical_length, max_vertical_length)
        for sid in semantics_ids
    )

    # set the semantics to -1 for the points that do not pass the filter
    for semantics_id, keep in results:
        if not keep:
            indices = np.where(points_semantics[:, 3] == semantics_id)[0]
            points_semantics[indices, 3] = -1

    # print the number of semantics_ids that passed the filter
    print(f"Number of semantics_ids that passed the size filter: {len(np.unique(points_semantics[:, 3]))-1}")
    # print the number of semantics_ids that did not pass the filter
    print(f"Number of semantics_ids that did not pass the size filter: {len(semantics_ids) - len(np.unique(points_semantics[:, 3]))+1}")
    return points_semantics


def pca_curvature_filter(points_semantics, curvature_threshold=0.5):
    """
    Filter points based on PCA planarity.
    :param points_semantics: numpy array of shape (N, 4) where N is the number of points
    :param curvature_threshold: float, threshold for curvature; large values indicate more objects will be removed
    """
    semantics_ids = np.unique(points_semantics[:, 3])

    def process_semantic_group(points_semantics, semantics_id, ratio):
        # Get indices of points with this semantics
        indices = np.where(points_semantics[:, 3] == semantics_id)[0]
        local_points_semantics = points_semantics[indices]
        xyz = local_points_semantics[:, :3]

        # Perform PCA
        pca = PCA(n_components=3)
        pca.fit(xyz)

        eigenvalues = pca.explained_variance_  # λ₁, λ₂, λ₃

        # Sort eigenvalues from largest to smallest
        eigenvalues = np.sort(eigenvalues)[::-1]
        λ1, λ2, λ3 = eigenvalues


        # Avoid divide by zero
        if λ1 == 0:
            return 0.0  # when all points are the same

        curvature = λ3 / (λ1 + λ2 + λ3)  
        
        if curvature > ratio:   # less flat
            return semantics_id, True
        else:  # more flat
            return semantics_id, False
        
    # Parallel processing
    results = Parallel(n_jobs=8)(
        delayed(process_semantic_group)(points_semantics, sid, curvature_threshold)
        for sid in semantics_ids
    )
    # set the semantics to -1 for the points that do not pass the filter
    for semantics_id, keep in results:
        if not keep:
            indices = np.where(points_semantics[:, 3] == semantics_id)[0]
            points_semantics[indices, 3] = -1
    # print the number of semantics_ids that passed the filter
    print(f"Number of semantics_ids that passed the PCA filter: {len(np.unique(points_semantics[:, 3]))-1}")
    # print the number of semantics_ids that did not pass the filter
    print(f"Number of semantics_ids that did not pass the PCA filter: {len(semantics_ids) - len(np.unique(points_semantics[:, 3]))+1}")
    return points_semantics

def density_filter(points_semantics, semantics_id, radius=0.1, density_threshold=0.5):
    semantics_ids = np.unique(points_semantics[:, 3])
    
    def compute_point_density(points_semantics, semantics_id, radius=0.1, density_threshold=0.5):
        # Get indices of points with this semantics
        indices = np.where(points_semantics[:, 3] == semantics_id)[0]
        local_points_semantics = points_semantics[indices]
        xyz = local_points_semantics[:, :3]
        # Create Open3D point cloud
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(xyz)
        # Compute point density
        pcd_tree = o3d.geometry.KDTreeFlann(pcd)
        densities = []
        points = np.asarray(pcd.points)
        
        for i in range(len(points)):
            [k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[i], radius)
            densities.append(k)

        density = np.mean(densities)
                
        if density >= density_threshold:
            return semantics_id, True
        else:
            return semantics_id, False
        
    # Parallel processing
    results = Parallel(n_jobs=8)(
        delayed(compute_point_density)(points_semantics, sid, radius, density_threshold)
        for sid in semantics_ids
    )
    # set the semantics to -1 for the points that do not pass the filter
    for semantics_id, keep in results:
        if not keep:
            indices = np.where(points_semantics[:, 3] == semantics_id)[0]
            points_semantics[indices, 3] = -1
    # print the number of semantics_ids that passed the filter
    print(f"Number of semantics_ids that passed the density filter: {len(np.unique(points_semantics[:, 3]))-1}")
    # print the number of semantics_ids that did not pass the filter
    print(f"Number of semantics_ids that did not pass the density filter: {len(semantics_ids) - len(np.unique(points_semantics[:, 3]))+1}")
    return points_semantics


def hwr_filter(points_semantics, hwr_threshold=10.0):
    semantics_ids = np.unique(points_semantics[:, 3])
    
    def compute_hwr(points_semantics, semantics_id):
        # Get indices of points with this semantics
        indices = np.where(points_semantics[:, 3] == semantics_id)[0]
        local_points_semantics = points_semantics[indices]
        xyz = local_points_semantics[:, :3]

        pca = PCA(n_components=3)
        pca.fit(xyz)

        # Get the eigenvalues
        eigenvalues = pca.explained_variance_
        # Get the eigenvectors
        eigenvectors = pca.components_

        # find the eigenvector aligned with the z-axis
        z_axis = np.array([0, 0, 1])
        # find the angle between the eigenvector and the z-axis
        angles = np.arccos(np.clip(np.dot(eigenvectors, z_axis), -1.0, 1.0))
        # find the index of the eigenvector aligned with the z-axis
        index = np.argmax(angles)
        # find the eigenvalue associated with the eigenvector aligned with the z-axis
        height = eigenvalues[index]
        # for the remaining eigenvalues, find the smaller one
        eigenvalues = np.delete(eigenvalues, index)
        width = np.min(eigenvalues)
        # compute the height to width ratio
        hwr = height / width
        return semantics_id, hwr
    
    # Parallel processing
    results = Parallel(n_jobs=8)(
        delayed(compute_hwr)(points_semantics, sid)
        for sid in semantics_ids if sid != -1
    )

    # set the semantics to -1 for the points that do not pass the filter
    for semantics_id, hwr in results:
        if hwr > hwr_threshold:
            print(semantics_id, hwr)
            indices = np.where(points_semantics[:, 3] == semantics_id)[0]
            points_semantics[indices, 3] = -1

    hwr_results = [hwr for _, hwr in results if hwr > hwr_threshold]
    print(hwr_results)

    # print the number of semantics_ids that passed the filter
    print(f"Number of semantics_ids that passed the HWR filter: {len(np.unique(points_semantics[:, 3]))-1}")
    # print the number of semantics_ids that did not pass the filter
    print(f"Number of semantics_ids that did not pass the HWR filter: {len(semantics_ids) - len(np.unique(points_semantics[:, 3]))+1}")
    return points_semantics


In [109]:
points_semantics_source, colors = read_las_file("data/centennial_bluff_mission_a_0.las")
points_semantics = points_semantics_source.copy()
points_semantics = cluster_filter(points_semantics, eps=0.3, min_samples=10, n_jobs=8)
points_semantics = size_filter(points_semantics, min_horizontal_length=0.3, max_horizontal_length=4.0, min_vertical_length=0.3, max_vertical_length=4.0)
save_points_to_las(points_semantics, colors, "data/centennial_bluff_mission_a_0_size_filtered.las")

points_semantics_source, colors = read_las_file("data/centennial_bluff_mission_a_0_size_filtered.las")
points_semantics = points_semantics_source.copy()
points_semantics = pca_curvature_filter(points_semantics, curvature_threshold=0.15)
save_points_to_las(points_semantics, colors, "data/centennial_bluff_mission_a_0_pca_filtered.las")


points_semantics_source, colors = read_las_file("data/centennial_bluff_mission_a_0_pca_filtered.las")
points_semantics = points_semantics_source.copy()
points_semantics = density_filter(points_semantics, semantics_id=1, radius=0.1, density_threshold=4.6)
save_points_to_las(points_semantics, colors, "data/centennial_bluff_mission_a_0_density_filtered.las")

points_semantics_source, colors = read_las_file("data/centennial_bluff_mission_a_0_density_filtered.las")
points_semantics = points_semantics_source.copy()
points_semantics = hwr_filter(points_semantics, hwr_threshold=4.0)
save_points_to_las(points_semantics, colors, "data/centennial_bluff_mission_a_0_hwr_filtered.las")


(4648369, 4)
Number of points: 4648369
Number of semantics: 6658
Number of outliers from DBSCAN filter: 264077
Number of semantics_ids that passed the size filter: 5845
Number of semantics_ids that did not pass the size filter: 814
Max intensity: 6568
(4648369, 4)
Number of points: 4648369
Number of semantics: 5846
Number of semantics_ids that passed the PCA filter: 93
Number of semantics_ids that did not pass the PCA filter: 5753
Max intensity: 6518
(4648369, 4)
Number of points: 4648369
Number of semantics: 94
Number of semantics_ids that passed the density filter: 44
Number of semantics_ids that did not pass the density filter: 50
Max intensity: 6519
(4648369, 4)
Number of points: 4648369
Number of semantics: 44
[]
Number of semantics_ids that passed the HWR filter: 43
Number of semantics_ids that did not pass the HWR filter: 1
Max intensity: 6519


In [67]:
def compute_point_density(points_semantics, semantics_id, radius=0.1, density_threshold=0.5):
    # Get indices of points with this semantics
    indices = np.where(points_semantics[:, 3] == semantics_id)[0]
    local_points_semantics = points_semantics[indices]
    xyz = local_points_semantics[:, :3]
    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    # Compute point density
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    densities = []
    points = np.asarray(pcd.points)
    
    for i in range(len(points)):
        [k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[i], radius)
        densities.append(k)

    density = np.mean(densities)

    print(f"Density: {density}")
            
    if density >= density_threshold:
        return semantics_id, True
    else:
        return semantics_id, False

In [73]:
compute_point_density(points_semantics, semantics_id=5932, radius=0.1, density_threshold=4.5)

Density: 5.005702066999287


(5932, True)